# Use MMLMTools with langchain

import needed packages

In [ ]:
from langchain import OpenAI
from langchain.agents import initialize_agent
from langchain.agents import Tool
from langchain.chains.conversation.memory import ConversationBufferMemory
from mmlmtools.apis.agents.visual_chatgpt import load_tools_for_visual_chatgpt

In [ ]:
# load mmlmtools
globals().update(load_tools_for_visual_chatgpt())

prepare the tools

In [ ]:
# assign the tool you want to use and its device
load_dict={'ImageCaptionTool': 'cpu'}

print(f'Initializing tools,load_dict={load_dict}')
models = {}
for class_name, device in load_dict.items():
    models[class_name] = globals()[class_name](device=device)

print(f'All the Available Functions: {models}')

tools = []
for instance in models.values():
    for e in dir(instance):
        if e.startswith('inference'):
            func = getattr(instance, e)
            tools.append(
                Tool(
                    name=func.name,
                    description=func.description,
                    func=func))
llm = OpenAI(temperature=0)
memory = ConversationBufferMemory(
    memory_key='chat_history', output_key='output')

agent = initialize_agent(
    tools=tools,
    llm=llm,    
    agent='conversational-react-description',
    verbose=True,
    memory=memory,
)

run the tool

In [ ]:
# give your input here
input_path = ''
agent.run(input=input_path)